# Environment

In [1]:
import warnings

In [2]:
warnings.simplefilter('ignore', FutureWarning)
warnings.simplefilter('ignore', UserWarning)

In [3]:
import re

In [4]:
import pandas as pd

In [5]:
import numpy as np

In [6]:
from math import log10, log2

In [7]:
import plotly.express as px

In [8]:
import plotly.graph_objects as go

In [9]:
import matplotlib.pyplot as plt

In [10]:
from flatsplode import flatsplode

In [11]:
from biothings_client import get_client

In [12]:
import gseapy as gp

# GSEApy

## Enrichr API

In [13]:
human_library = gp.get_library_name(organism="Human")

In [14]:
human_library

['ARCHS4_Cell-lines',
 'ARCHS4_IDG_Coexp',
 'ARCHS4_Kinases_Coexp',
 'ARCHS4_TFs_Coexp',
 'ARCHS4_Tissues',
 'Achilles_fitness_decrease',
 'Achilles_fitness_increase',
 'Aging_Perturbations_from_GEO_down',
 'Aging_Perturbations_from_GEO_up',
 'Allen_Brain_Atlas_10x_scRNA_2021',
 'Allen_Brain_Atlas_down',
 'Allen_Brain_Atlas_up',
 'Azimuth_2023',
 'Azimuth_Cell_Types_2021',
 'BioCarta_2013',
 'BioCarta_2015',
 'BioCarta_2016',
 'BioPlanet_2019',
 'BioPlex_2017',
 'CCLE_Proteomics_2020',
 'CORUM',
 'COVID-19_Related_Gene_Sets',
 'COVID-19_Related_Gene_Sets_2021',
 'Cancer_Cell_Line_Encyclopedia',
 'CellMarker_Augmented_2021',
 'ChEA_2013',
 'ChEA_2015',
 'ChEA_2016',
 'ChEA_2022',
 'Chromosome_Location',
 'Chromosome_Location_hg19',
 'ClinVar_2019',
 'DSigDB',
 'Data_Acquisition_Method_Most_Popular_Genes',
 'DepMap_WG_CRISPR_Screens_Broad_CellLines_2019',
 'DepMap_WG_CRISPR_Screens_Sanger_CellLines_2019',
 'Descartes_Cell_Types_and_Tissue_2021',
 'Diabetes_Perturbations_GEO_2022',
 'DisG

In [15]:
GO_Biological_Process_2023 = gp.get_library(name="GO_Biological_Process_2023",
                                            organism="Human")

In [17]:
type(GO_Biological_Process_2023)

dict

In [53]:
len(GO_Biological_Process_2023)

5406

In [54]:
GO_Cellular_Component_2023 = gp.get_library(name="GO_Cellular_Component_2023",
                                            organism="Human")

In [55]:
type(GO_Cellular_Component_2023)

dict

In [56]:
len(GO_Cellular_Component_2023)

472

In [57]:
GO_Molecular_Function_2023 = gp.get_library(name="GO_Molecular_Function_2023",
                                            organism="Human")

In [58]:
type(GO_Molecular_Function_2023)

dict

In [59]:
len(GO_Molecular_Function_2023)

1147

In [15]:
from sqlalchemy import create_engine, Column, String, Integer
from sqlalchemy.orm import declarative_base, sessionmaker

In [16]:
Base = declarative_base()

In [17]:
class GOBP(Base):
    __tablename__ = 'GOBP'
    id = Column(Integer, primary_key=True)
    Term = Column(String)
    Gene = Column(String)

In [18]:
class GOCC(Base):
    __tablename__ = 'GOCC'
    id = Column(Integer, primary_key=True)
    Term = Column(String)
    Gene = Column(String)

In [19]:
class GOMF(Base):
    __tablename__ = 'GOMF'
    id = Column(Integer, primary_key=True)
    Term = Column(String)
    Gene = Column(String)

In [21]:
engine = create_engine('sqlite:///instance/GeneSet.db')
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

In [89]:
for Term, Genes in GO_Biological_Process_2023.items():
    for Gene in Genes:
        new_entry = GOBP(Term=Term, Gene=Gene)
        session.add(new_entry)

In [90]:
for Term, Genes in GO_Cellular_Component_2023.items():
    for Gene in Genes:
        new_entry = GOCC(Term=Term, Gene=Gene)
        session.add(new_entry)

In [91]:
for Term, Genes in GO_Molecular_Function_2023.items():
    for Gene in Genes:
        new_entry = GOMF(Term=Term, Gene=Gene)
        session.add(new_entry)

In [92]:
session.commit()
session.close()

In [22]:
gp_records = session.query(GOBP).all()

session.close()

In [23]:
geneset_dict = {}

for record in gp_records:
    if record.Term in geneset_dict:
        geneset_dict[record.Term].append(record.Gene)
    else:
        geneset_dict[record.Term] = [record.Gene]

In [24]:
# geneset_dict

In [25]:
input_genes = "UBL5,NDUFB8,CHMP2B,PRPF8,NOSTRIN,MFAP1,CWC22,PLCH2,PRPF31,ATP6AP1,DSC3,CLN5,CHDC2,PIP,ZNF473,DHX8,RAB5A,NUP98,NUPL1,HRK,SLC41A3,SNRPD3,SNRPD2,PCGF6,GSK3A,SLC2A2,ARCN1,ANGPTL3,COPB1,COPB2,STARD5,CYB5R4,DMD,FUS,COPS6,NOS3,SFXN2,CNTN5,IQSEC1,CRADD,STAC3,COPZ1,SULT1C4,EFTUD2,SLC35A1,B4GALT2,THRSP,NHP2L1,ZNF224,NXF1,PDLIM3,SAMM50,MTFR1,SART1,PCDHGA1,GINS2,MPG,GJA3,UBE2A,ESAM,CLK3,EIF4A2,NUTF2,INMT,SCYL3,XIAP,TRIM28,MPZL1,LY6G6C,RBM14,TPRX1,ATP6V0B,NUP107,ABCD1,TGS1,RPS4X,CRNKL1,YTHDC1,PPAN,PRRT1,KRTCAP2,GDPD5,ZNF132,WDR18,ATP5F1,AMOTL1,RPS16,CCDC74B,CCDC74A,SF3B1,SF3B3,SF3B2,MYOCD,MLKL,PRSS8,CACTIN,EIF2S1,ZNF16,PGD,SRP54,AQR,DYNC1I1,DCLRE1B,CALCOCO2,EVC2,LRP1B,ZNF552,COPG1,EPRS,COPA,ATP6V1G1,PIEZO1,FCGR2A,CPLX1,SNRPB,BACE1,ZNF154,RAB29,ATP6V0E2,AHCY,SLC1A3"

In [26]:
gene_list = [
    i for i in re.split(r"\n|\t|,|;| ", input_genes) if i != ""
]

In [27]:
len(gene_list)

121

In [28]:
gene_list[0:5]

['UBL5', 'NDUFB8', 'CHMP2B', 'PRPF8', 'NOSTRIN']

In [29]:
# help(gp.enrich)

In [30]:
enr = gp.enrich(gene_list=gene_list,
                gene_sets=geneset_dict,
                background=None)

In [34]:
    gp_results = (
        enr.results.filter(
            [
                # "Gene_set",
                "Term",
                "Overlap",
                "P-value",
                "Adjusted P-value",
                "Odds Ratio",
                "Combined Score",
                "Genes",
            ]
        )
        .rename(
            columns={
                "P-value": "p_val",
                "Adjusted P-value": "adj_p",
                "Odds Ratio": "OddsRatio",
                "Combined Score": "Score",
            }
        )
        .sort_values("p_val")
        # .query(f'adj_p < {cutoff}')
        .query("p_val < 0.05")
        .assign(Term_ed=lambda df: df.Term.str.replace(" \\(GO:.*\\)$", "", regex=True))
        .assign(mLog10_p_val=lambda df: df.p_val.apply(log10))
        .assign(mLog10_p_val=lambda df: df.mLog10_p_val * -1)
        .assign(mLog10_adj_p=lambda df: df.adj_p.apply(log10))
        .assign(mLog10_adj_p=lambda df: df.mLog10_adj_p * -1)
        .assign(num_Score=lambda df: df.Score)
        .assign(Log2_Score=lambda df: df.Score.apply(log2))
        .assign(Genes=lambda df: df.Genes.str.replace(";", "; "))
        .assign(p_val=lambda df: df.p_val.apply(lambda x: f"{x:.3e}"))
        .assign(adj_p=lambda df: df.adj_p.apply(lambda x: f"{x:.3e}"))
        .assign(OddsRatio=lambda df: df.OddsRatio.apply(lambda x: f"{x:.3f}"))
        .assign(Score=lambda df: df.Score.apply(lambda x: f"{x:.3f}"))
    )

In [36]:
gp_results.head()

,Term,Overlap,p_val,adj_p,OddsRatio,Score,Genes,Term_ed,mLog10_p_val,mLog10_adj_p,num_Score,Log2_Score
926,"mRNA Splicing, Via Spliceosome (GO:0000398)",19/211,2.470e-16,1.176e-13,16.190,581.806,YTHDC1; SNRPB; CACTIN; SNRPD2; PRPF31; TGS1; D...,"mRNA Splicing, Via Spliceosome",15.607348,12.929694,581.806476,9.184396
640,"RNA Splicing, Via Transesterification Reaction...",18/180,2.512e-16,1.176e-13,17.820,640.086,CRNKL1; RBM14; YTHDC1; PRPF8; MFAP1; EFTUD2; S...,"RNA Splicing, Via Transesterification Reaction...",15.599940,12.929694,640.086234,9.322122
924,mRNA Processing (GO:0006397),18/214,5.432e-15,1.695e-12,14.960,491.382,CRNKL1; RBM14; YTHDC1; PRPF8; MFAP1; EFTUD2; S...,mRNA Processing,14.265004,11.770849,491.381793,8.940701
862,Spliceosomal snRNP Assembly (GO:0000387),7/37,5.099e-09,1.193e-06,31.028,592.446,PRPF8; SNRPB; SART1; PRPF31; TGS1; SNRPD3; SNRPD2,Spliceosomal snRNP Assembly,8.292507,5.923291,592.445633,9.210539
895,U2-type Prespliceosome Assembly (GO:1903241),6/23,8.325e-09,1.459e-06,42.502,790.701,SNRPB; SF3B2; SF3B3; SF3B1; SNRPD3; SNRPD2,U2-type Prespliceosome Assembly,8.079627,5.836059,790.701265,9.626989
